Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

Initializing variables

In [ ]:
x=np.random.rand(2,100)
d=np.zeros((100,1))
d1=np.zeros((100,1))
D=np.zeros((100,1))
Cp=[]
Cn=[]


Plotting the given data points

In [ ]:
X,Y=np.meshgrid(np.linspace(0,1,100),np.linspace(0,1,100))
sun=np.square(X-0.8)+np.square(Y-0.5)-np.square(0.15)
#plt.contour(Y,X,sun,[0])

y=np.linspace(0,1,100)
mount=((1/5)*np.sin(10*y))+0.3
#plt.plot(y,mount)

for i in range(len(x[0])):

    d[i][0]=((1/5)*np.sin(10*x[0][i]))+0.3
    d1[i][0]=np.square(x[1][i]-0.8)+np.square(x[0][i]-0.5)

    if x[1][i] < d[i][0] or d1[i][0] < np.square(0.15):
        D[i][0]=1
        Cp.append((x[0][i],x[1][i]))
        cp=np.asarray(Cp)
        plt.plot(x[0][i],x[1][i],'r.')

    else:
        D[i][0]=-1
        Cn.append((x[0][i],x[1][i]))
        cn=np.asarray(Cn)
        plt.plot(x[0][i],x[1][i],'b.')

Picking the centroid from the given set of datapoints

In [ ]:
index_p=np.random.choice(len(cp),10,replace=0)
print(index_p)

index_n=np.random.choice(len(cn),10,replace=0)
print(index_n)

In [ ]:
center_p=np.zeros((10,2))
center_n=np.zeros((10,2))
for i in range(10):
        center_p[i]=Cp[index_p[i]]
        center_n[i]=Cn[index_n[i]]

Plotting The chosen centrod points

In [ ]:
plt.plot(cp[:,0],cp[:,1],'r.')
plt.scatter(center_p[:,0],center_p[:,1],marker='*',c='y',s=150)
plt.plot(cn[:,0],cn[:,1],'b.')
plt.scatter(center_n[:,0],center_n[:,1],marker='o',c='g',s=50)

Finding the new cluster centers for the positive class

In [ ]:
k_p=10
center_p_old=np.zeros(center_p.shape)
center_p_new=center_p
cluster_p=np.zeros(len(cp))
dist_p=np.zeros((len(cp),k_p))
error_p = np.linalg.norm(center_p_new - center_p_old)

while error_p != 0:
    # Measure the distance to every center
    for i in range(k_p):
        dist_p[:,i] = np.linalg.norm(cp - center_p_new[i], axis=1)
    # Assign all training data to closest center
    cluster_p = np.argmin(dist_p, axis = 1)
    
    center_p_old = center_p_new
    # Calculate mean for every cluster and update the center
    for i in range(k_p):
        center_p_new[i] = np.mean(cp[cluster_p == i], axis=0)
    error_p = np.linalg.norm(center_p_new - center_p_old)
print('New cluster centers:',center_p_new)
print('Updated clusters',cluster_p)


Finding the new cluster centers for the negative class

In [ ]:
k_n=10
center_n_old=np.zeros(center_n.shape)
center_n_new=center_n
cluster_n=np.zeros(len(cn))
dist_n=np.zeros((len(cn),k_n))
error_n = np.linalg.norm(center_n_new - center_n_old)

while error_n != 0:
    # Measure the distance to every center
    for i in range(k_n):
        dist_n[:,i] = np.linalg.norm(cn - center_n_new[i], axis=1)
    # Assign all training data to closest center
    cluster_n = np.argmin(dist_n, axis = 1)
    
    center_n_old = center_n_new
    # Calculate mean for every cluster and update the center
    for i in range(k_n):
        center_n_new[i] = np.mean(cn[cluster_n == i], axis=0)
    error_n = np.linalg.norm(center_n_new - center_n_old)
print('New cluster centers:',center_n_new)
print('Updated clusters',cluster_n)


Plotting the updated centroids

In [ ]:
plt.plot(cp[:,0],cp[:,1],'r.')
plt.scatter(center_p_new[:,0],center_p_new[:,1],marker='*',c='y',s=150)
plt.plot(cn[:,0],cn[:,1],'b.')
plt.scatter(center_n[:,0],center_n[:,1],marker='o',c='g',s=50)

Concatenating the centers of both positive and negative classes

In [ ]:
center=np.row_stack((center_p_new,center_n_new))
print(center)

Computing the RBF function

In [ ]:
x_mu=np.zeros((100,len(center)))
for i in range(100):
    for j in range(len(center)):
        x_mu[i][j]=np.linalg.norm(x[:,i]-center[j,:])
rbf=np.exp(-16*np.square(x_mu))

Perceptron training algorithm

In [ ]:
K=20
W=np.zeros((K,1))
W=np.random.normal(size=(K,1))              #Initializing the weight vector 
bias=np.random.randn(1)
errors=0                              #Initializing the misclassifications
Mc=[]                                 #Storing the misclassification values after every epoch
epoch=0
lr=0.01
#rbf=np.row_stack([rbf_p,rbf_n])


In [ ]:
while(epoch==0 or Mc[epoch-1] > 0):
#     errors=0
    for i in range(len(rbf)):
        #errors=0
        y=np.sign(np.matmul(W.reshape(1,20),rbf[i].reshape(20,1))+bias)        
        #Y=np.heaviside(y,1)
        print('actual',y)
        E=D[i]-y
        print('Error',E)
        Q=E*rbf[i].reshape(20,1)
        W=W+lr*Q  
        bias=bias+lr*E
        errors = 0
        for j in range(len(rbf)):
            Y=np.sign(np.matmul(W.reshape(1,20),rbf[j].reshape(20,1))+bias) 
            if Y != D[j]:
                errors=errors+1
#                 print('count',errors)

        Mc.append(errors)
        print('Misclassification',Mc[epoch])
        epoch=epoch+1
    
    

Plotting Number of epochs against Misclassifications

In [ ]:
plt.plot(Mc)
plt.xlabel('Number of epochs')
plt.ylabel('Number of misclassifications')
plt.title('Epochs vs Misclassification')
plt.show()

Plotting the Decision Boundary

In [ ]:
plt.plot(cp[:,0],cp[:,1],'r.')
plt.scatter(center_p_new[:,0],center_p_new[:,1],marker='*',c='y',s=150)
plt.plot(cn[:,0],cn[:,1],'b.')
plt.scatter(center_n[:,0],center_n[:,1],marker='o',c='g',s=50)
K_mu=0

for i in np.linspace(0,1,1000):
    for j in np.linspace(0,1,1000):
        g=0
        for k in range(len(center)):
            K_mu=np.linalg.norm(np.asarray([i,j])-center[k,:])
            RBF=np.exp(-16*np.square(K_mu))
            g=g+np.multiply(W[k],RBF)
        g=g+bias
        print('G:',g)
        if np.abs(g)<0.005:
            plt.scatter(i,j,c='g')
                    
            